In [1]:
# Importación de librerías
import os
import numpy as np
import time
import serial
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
categorias = ['vidrio', 'carton', 'metal', 'plastico']

In [3]:
!pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('modelo2.h5')

img_path = 'metal.jpg'
img = image.load_img(img_path, target_size=(32, 32))

img_array = image.img_to_array(img)

img_array = np.expand_dims(img_array, axis=0)

img_array /= 255.0

prediccion = model.predict(img_array)

print(prediccion)

categoriaM = np.argmax(prediccion)

print(f'El índice de la categoría predicha es: {categoriaM} con una probabilidad de {prediccion[0][categoriaM]:.2f}')

In [ ]:
!pip install  tensorflow

In [ ]:
!pip install pyserial

In [ ]:
# Obtener el tiempo actual (en milisegundos)
def tiempo():
    return int(round(time.time() * 1000))

In [6]:
# Conexión conexión con el puerto serial
COM = 'COM13'
BAUD = '115200'
ser = serial.Serial(COM, BAUD)

contador_imagenes = {categoria: 0 for categoria in categorias}

# Umbral de probabilidad máxima
umbral_probabilidad = 0.9

# Obtener el tiempo actual
ultima_clasificacion = tiempo()

# Inicializar la captura de video (0 para la cámara por defecto)
camara = cv2.VideoCapture(0)

# Proceso de clasificación
while True:

    # Comprobar si hay datos en espera
    if ser.in_waiting == 0:
        continue

    # Leer el dato del puerto serial
    dato = ser.readline().decode('utf-8').rstrip()
    print(dato)

    if dato != "1":
        continue

    # Conectar con la cámara
    retorno, frame = camara.read()

    # Verificar si el frame fue capturado correctamente
    if not retorno:
        break

            if not ret:
                break

    # Comprobar si han pasado 5 segundos desde la última clasificación
    if (tiempo() - ultima_clasificacion) > 5000:
        
        # Hacer la predicción
        prediccion = modelo.predict(imagen_arreglo)
        categoria = np.argmax(prediccion)
        probabilidad_maxima = np.max(prediccion)
        ultima_clasificacion = tiempo()

        nombre_categoria = categorias[categoria]

        # Verificar si la probabilidad máxima supera el umbral
        if probabilidad_maxima >= umbral_probabilidad:
            # Guardar la imagen en la carpeta correspondiente
            contador_imagenes[nombre_categoria] += 1
            nombre_imagen = f"{nombre_categoria}_{contador_imagenes[nombre_categoria]}.png"
            ruta_imagen = os.path.join(nombre_categoria, nombre_imagen)
            # Guardar la imagen original (frame)
            cv2.imwrite(ruta_imagen, frame)

        ser.write(f'%{categoria};\n'.encode())
    
    cv2.putText(frame, f'Categoria: {categoria}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Precisión: {probabilidad_maxima}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('NeuroEcoWaste Solutions', frame)
    print(f'Predicción: {prediccion}')

    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()


..Conexión exitosa a ThingsBoard

 Conectado
Mensaje: Rancés - 000444

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
Predicción: [[0.8816516  0.00418125 0.00423634 0.10993085]]
Mensaje: Rancés - 000333

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicción: [[9.9997759e-01 3.0629570e-07 8.9716332e-07 2.1209466e-05]]
Mensaje: Rancés - 000444

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicción: [[9.9456006e-01 1.3135598e-05 8.1707221e-06 5.4185903e-03]]
